# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [1]:
# YOUR CHANGES HERE

# Setup
import pandas as pd
import numpy as np

def pick(opts, cols):
    for c in opts:
        if c in cols:
            return c
    raise ValueError(f"Missing one of {opts}; found {list(cols)}")

trans = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rew0  = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

trans.columns = [c.strip().lower() for c in trans.columns]
rew0.columns  = [c.strip().lower() for c in rew0.columns]

s_col  = pick(["state","s"], trans.columns)
a_col  = pick(["action","a"], trans.columns)
sp_col = pick(["next_state","next","sprime","s_next","sp"], trans.columns)
p_col  = pick(["prob","probability","p"], trans.columns)

trans = trans[[s_col, a_col, sp_col, p_col]].copy()
trans[s_col]  = trans[s_col].astype(str)
trans[a_col]  = trans[a_col].astype(str)
trans[sp_col] = trans[sp_col].astype(str)
trans[p_col]  = trans[p_col].astype(float)

states = sorted(set(trans[s_col]).union(set(trans[sp_col])))

group = {}
actions_by_state = {}
for (s, a), df in trans.groupby([s_col, a_col]):
    group[(s, a)] = list(zip(df[sp_col].tolist(), df[p_col].tolist()))
    actions_by_state.setdefault(s, set()).add(a)

def build_reward_fn(rew_df):
    r = rew_df.copy()
    r.columns = [c.strip().lower() for c in r.columns]
    if "reward" not in r.columns:
        raise ValueError("Rewards must have a 'reward' column.")

    if "next_state" not in r.columns:
        for alt in ["next","sprime","s_next","sp"]:
            if alt in r.columns:
                r = r.rename(columns={alt:"next_state"})
                break

    r_sas = None
    r_sa  = None
    r_s   = None

    if ("state" in r.columns) and ("action" in r.columns) and ("next_state" in r.columns):
        r_sas = {(str(row["state"]), str(row["action"]), str(row["next_state"])): float(row["reward"])
                 for _, row in r.iterrows()}
    elif ("state" in r.columns) and ("action" in r.columns):
        r_sa = {(str(row["state"]), str(row["action"])): float(row["reward"])
                for _, row in r.iterrows()}
    elif "state" in r.columns:
        r_s = {str(row["state"]): float(row["reward"]) for _, row in r.iterrows()}
    else:
        raise ValueError("Rewards must contain at least 'state'.")

    def reward(s, a, sp):
        if r_sas is not None:
            return r_sas.get((s, a, sp), 0.0)
        if r_sa is not None:
            return r_sa.get((s, a), 0.0)
        return r_s.get(sp, r_s.get(s, 0.0))

    return reward

reward_discomfort = build_reward_fn(rew0)


def evaluate_policy_value(policy_map, reward_fn, gamma=0.99, tol=1e-10, max_iter=200000):
    V = {s: 0.0 for s in states}
    for _ in range(max_iter):
        delta = 0.0
        Vnew = {}
        for s in states:
            a = policy_map.get(s)
            if a is None or (s, a) not in group:
                Vnew[s] = V[s]
                continue
            v = 0.0
            for sp, p in group[(s, a)]:
                v += p * (reward_fn(s, a, sp) + gamma * V.get(sp, 0.0))
            Vnew[s] = v
            delta = max(delta, abs(v - V[s]))
        V = Vnew
        if delta < tol:
            break
    return V

def value_iteration_optimal_policy(reward_fn, gamma=0.99, tol=1e-10, max_iter=200000):
    V = {s: 0.0 for s in states}
    for _ in range(max_iter):
        delta = 0.0
        Vnew = {}
        for s in states:
            acts = actions_by_state.get(s)
            if not acts:
                Vnew[s] = V[s]
                continue
            best = -1e300
            for a in acts:
                v = 0.0
                for sp, p in group[(s, a)]:
                    v += p * (reward_fn(s, a, sp) + gamma * V.get(sp, 0.0))
                if v > best:
                    best = v
            Vnew[s] = best
            delta = max(delta, abs(best - V[s]))
        V = Vnew
        if delta < tol:
            break

    policy = {}
    for s in states:
        acts = actions_by_state.get(s)
        if not acts:
            continue
        best_a = None
        best_q = -1e300
        for a in acts:
            q = 0.0
            for sp, p in group[(s, a)]:
                q += p * (reward_fn(s, a, sp) + gamma * V.get(sp, 0.0))
            if q > best_q:
                best_q = q
                best_a = a
        policy[s] = best_a
    return policy, V

# part 1
policy_do_nothing = {s: "do-nothing" for s in states}
V_dn = evaluate_policy_value(policy_do_nothing, reward_discomfort, gamma=0.99)
do_nothing_discomfort = {s: -V_dn[s] for s in states}

Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [2]:
# YOUR CHANGES HERE

pd.DataFrame({"state": states,
              "expected_discomfort": [do_nothing_discomfort[s] for s in states]}
).to_csv("do-nothing-discomfort.tsv", sep="\t", index=False)

Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [3]:
# YOUR CHANGES HERE


#trans = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
#rew = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

#policy_discomfort, _ = value_iteration_optimal_policy(reward_discomfort, gamma=0.99)

gamma = 1.0
tol = 1e-12
max_iter = 200000

def pick(opts, cols):
    for c in opts:
        if c in cols:
            return c
    raise ValueError(f"Missing one of {opts}; found {list(cols)}")

s_col  = pick(["state","s"], trans.columns)
a_col  = pick(["action","a"], trans.columns)
sp_col = pick(["next_state","next","sprime","s_next","sp"], trans.columns)
p_col  = pick(["prob","probability","p"], trans.columns)

trans[s_col]  = trans[s_col].astype(str)
trans[a_col]  = trans[a_col].astype(str)
trans[sp_col] = trans[sp_col].astype(str)
trans[p_col]  = trans[p_col].astype(float)

states = sorted(set(trans[s_col]).union(set(trans[sp_col])))

group = {}
actions_by_state = {}
for (s, a), df in trans.groupby([s_col, a_col]):
    group[(s, a)] = list(zip(df[sp_col].tolist(), df[p_col].tolist()))
    actions_by_state.setdefault(s, set()).add(a)

def is_terminal_state(s):
    acts = actions_by_state.get(s, None)
    return (not acts)

V = {s: 0.0 for s in states}

for _ in range(max_iter):
    delta = 0.0
    Vnew = {}
    for s in states:
        if is_terminal_state(s):
            Vnew[s] = 0.0
            continue
        best = -1e300
        for a in actions_by_state[s]:
            v = 0.0
            for sp, p in group[(s, a)]:
                v += p * (reward_discomfort(s, a, sp) + gamma * V.get(sp, 0.0))
            if v > best:
                best = v
        Vnew[s] = best
        delta = max(delta, abs(best - V[s]))
    V = Vnew
    if delta < tol:
        break

policy_discomfort = {}
for s in states:
    acts = actions_by_state.get(s, None)
    if not acts:
        continue
    best_a = None
    best_q = -1e300
    for a in acts:
        q = 0.0
        for sp, p in group[(s, a)]:
            q += p * (reward_discomfort(s, a, sp) + gamma * V.get(sp, 0.0))
        if q > best_q:
            best_q = q
            best_a = a
    policy_discomfort[s] = best_a

Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [4]:
# YOUR CHANGES HERE
pd.DataFrame({"state": states,
              "action": [policy_discomfort.get(s, "") for s in states]}
).to_csv("minimum-discomfort-actions.tsv", sep="\t", index=False)

Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [5]:
# YOUR CHANGES HERE

Pi = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t")
Pi_map = dict(zip(Pi["state"].astype(str), Pi["action"].astype(str)))

def terminal_under_policy(s):
    a = Pi_map.get(s, None)
    if a is None or (s, a) not in group:
        return True
    nxt = group[(s, a)]
    if len(nxt) == 1 and nxt[0][0] == s and abs(nxt[0][1] - 1.0) < 1e-12 and reward_discomfort(s, a, s) == 0.0:
        return True
    return False

nonterm = [s for s in states if not terminal_under_policy(s)]
term = [s for s in states if terminal_under_policy(s)]

idx = {s: i for i, s in enumerate(nonterm)}
n = len(nonterm)

A = np.zeros((n, n), dtype=float)
b = np.zeros(n, dtype=float)

for s in nonterm:
    i = idx[s]
    a = Pi_map[s]
    A[i, i] = 1.0
    rhs = 0.0
    for sp, p in group[(s, a)]:
        rhs += p * reward_discomfort(s, a, sp)
        if sp in idx:
            A[i, idx[sp]] -= p  # gamma=1
    b[i] = rhs

# Solve; if still singular (rare), fall back to least-squares
try:
    V_nonterm = np.linalg.solve(A, b)
except np.linalg.LinAlgError:
    V_nonterm = np.linalg.lstsq(A, b, rcond=None)[0]

V = {s: 0.0 for s in term}
for s in nonterm:
    V[s] = float(V_nonterm[idx[s]])

expected_discomfort = {s: -V[s] for s in states}

Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [6]:
# YOUR CHANGES HERE
pd.DataFrame({
    "state": states,
    "expected_discomfort": [expected_discomfort[s] for s in states]
}).to_csv("minimum-discomfort-values.tsv", sep="\t", index=False)

Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick (must have symptoms, exposed does not count) and 0 otherwise.
To be clear, the action does not matter for this reward function.


In [7]:
# YOUR CHANGES HERE

duration_rewards = rew0.copy()
duration_rewards.columns = [c.strip().lower() for c in duration_rewards.columns]

def is_sick(state):
    x = str(state).lower()
    return ("symptom" in x) or ("fever" in x) or ("sick" in x)

duration_rewards["reward"] = duration_rewards["state"].apply(lambda s: -1 if is_sick(s) else 0)
reward_duration = build_reward_fn(duration_rewards)

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [8]:
# YOUR CHANGES HERE
duration_rewards.to_csv("duration-rewards.tsv", sep="\t", index=False)

Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [9]:
# YOUR CHANGES HERE

policy_speed, _ = value_iteration_optimal_policy(reward_duration, gamma=0.99)

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [10]:
# YOUR CHANGES HERE

pd.DataFrame({"state": states,
              "action": [policy_speed.get(s, "") for s in states]}
).to_csv("minimum-duration-actions.tsv", sep="\t", index=False)

Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [11]:
# YOUR CHANGES HERE
def evaluate_policy_expected_steps(policy_map, step_cost_by_state, tol=1e-10, max_iter=200000):
    D = {s: 0.0 for s in states}
    for _ in range(max_iter):
        delta = 0.0
        Dnew = {}
        for s in states:
            a = policy_map.get(s)
            if a is None or (s, a) not in group:
                Dnew[s] = D[s]
                continue
            v = float(step_cost_by_state.get(s, 0.0))
            for sp, p in group[(s, a)]:
                v += p * D.get(sp, 0.0)
            Dnew[s] = v
            delta = max(delta, abs(v - D[s]))
        D = Dnew
        if delta < tol:
            break
    return D

r_map = {str(row["state"]): float(row["reward"]) for _, row in duration_rewards.iterrows()}
step_cost = {s: (1.0 if r_map.get(s, 0.0) == -1 else 0.0) for s in states}

expected_sick_days = evaluate_policy_expected_steps(policy_speed, step_cost)

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [12]:
# YOUR CHANGES HERE

pd.DataFrame({"state": states,
              "expected_sick_days": [expected_sick_days[s] for s in states]}
).to_csv("minimum-duration-days.tsv", sep="\t", index=False)

Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [13]:
# YOUR CHANGES HERE

V_speed_disc  = evaluate_policy_value(policy_speed, reward_discomfort, gamma=0.99)
V_pamper_disc = evaluate_policy_value(policy_discomfort, reward_discomfort, gamma=0.99)

speed_discomfort = {s: -V_speed_disc[s] for s in states}
pamper_discomfort = {s: -V_pamper_disc[s] for s in states}

Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [14]:
# YOUR CHANGES HERE

pd.DataFrame({
    "state": states,
    "speed_discomfort": [speed_discomfort[s] for s in states],
    "minimize_discomfort": [pamper_discomfort[s] for s in states],
}).to_csv("policy-comparison.tsv", sep="\t", index=False)

Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.